# D18 MultiIndex

## Mục đích

Chính thức giới thiệu với các bạn các thao tác liên quan tới index nhiều cấp.


## Tạo index nhiều cấp

Dữ liệu của index nhiều cấp được lưu trữ trong một lớp riêng trong Pandas tên là `MultiIndex`. Có nhiều cách để tạo dữ liệu nhiều cấp từ các cấu trúc dữ liệu khác nhau.

In [1]:
import pandas as pd

# Dùng tuple
mi_tuple = ("Nam", "Trẻ"), ("Nam", "Già"), ("Nữ", "Trẻ"), ("Nữ", "Già")
pd.MultiIndex.from_tuples(mi_tuple, names=["Giới", "Tuổi"])

MultiIndex([('Nam', 'Trẻ'),
            ('Nam', 'Già'),
            ( 'Nữ', 'Trẻ'),
            ( 'Nữ', 'Già')],
           names=['Giới', 'Tuổi'])

In [2]:
# Mỗi cấp là một danh sách
mi_array = [["Nam", "Nam", "Nữ", "Nữ"], ["Trẻ", "Già", "Trẻ", "Già"]]
pd.MultiIndex.from_arrays(mi_array, names=["Giới", "Tuổi"])

MultiIndex([('Nam', 'Trẻ'),
            ('Nam', 'Già'),
            ( 'Nữ', 'Trẻ'),
            ( 'Nữ', 'Già')],
           names=['Giới', 'Tuổi'])

In [3]:
# Kết hợp các danh mục giữa các cấp theo từng cặp
mi_levels = [["Nam", "Nữ"], ["Trẻ", "Già"]]
pd.MultiIndex.from_product(mi_levels, names=["Giới", "Tuổi"])

MultiIndex([('Nam', 'Trẻ'),
            ('Nam', 'Già'),
            ( 'Nữ', 'Trẻ'),
            ( 'Nữ', 'Già')],
           names=['Giới', 'Tuổi'])

## Truy cập chỉ mục khi có index nhiều cấp

Khi tên cột hoặc index của bạn là dạng nhiều cấp, việc truy cập sẽ phức tạp hơn một chút. Xem ví dụ.

In [4]:
d = pd.read_excel("../assets/hrm.xlsx")
d["age"] = d["date_exam"].dt.year - d["yob"]
d["bmi"] = d["weight"] / d["height"] ** 2
d["bmi_group"] = 1 + d["bmi"].ge(18) + d["bmi"].gt(23.5)
d["fssg_total"] = d.filter(like="q_fssg_").fillna(0).sum(axis=1).where(d["fssg"].eq("Y"))
d = d.drop(columns=["fssg"] + d.filter(like="q_fssg_").columns.to_list())

d_agg = d.groupby(["sex", "bmi_group"]).agg({
    "age": ["mean", "std"],
    "fssg_total": ["median", lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)],
    "eso_LA": lambda x: x.eq(0).sum() / x.count() * 100
})

d_agg

age            fssg_total                        \
                    mean        std     median <lambda_0> <lambda_1>   
sex bmi_group                                                          
0   1          48.040000  12.921171       14.0       10.0       18.0   
    2          46.981707  11.030617       13.0        9.0       19.0   
    3          50.171429  11.853234       13.0        9.0       16.5   
1   1          38.857143  10.415190       16.0       10.5       18.0   
    2          46.250000  11.998058       12.0        9.0       16.0   
    3          44.487179  10.101776       11.0        7.0       15.5   

                  eso_LA  
                <lambda>  
sex bmi_group             
0   1          70.833333  
    2          69.480519  
    3          75.862069  
1   1          42.857143  
    2          52.727273  
    3          38.888889

Hãy cùng xem index và tên cột của data frame này.

In [5]:
d_agg.index

MultiIndex([(0, 1),
            (0, 2),
            (0, 3),
            (1, 1),
            (1, 2),
            (1, 3)],
           names=['sex', 'bmi_group'])

In [6]:
d_agg.columns

MultiIndex([(       'age',       'mean'),
            (       'age',        'std'),
            ('fssg_total',     'median'),
            ('fssg_total', '<lambda_0>'),
            ('fssg_total', '<lambda_1>'),
            (    'eso_LA',   '<lambda>')],
           )

Chúng ta có thể truy cập vào một hàng cụ thể.

In [7]:
d_agg.loc[(0, 1)]

age         mean          48.040000
            std           12.921171
fssg_total  median        14.000000
            <lambda_0>    10.000000
            <lambda_1>    18.000000
eso_LA      <lambda>      70.833333
Name: (0, 1), dtype: float64

Hoặc một cột cụ thể.

In [8]:
d_agg[("age", "mean")]

sex  bmi_group
0    1            48.040000
     2            46.981707
     3            50.171429
1    1            38.857143
     2            46.250000
     3            44.487179
Name: (age, mean), dtype: float64

Và slicing.

In [9]:
d_agg.loc[(0, 1):(1, 1)]

age            fssg_total                        \
                    mean        std     median <lambda_0> <lambda_1>   
sex bmi_group                                                          
0   1          48.040000  12.921171       14.0       10.0       18.0   
    2          46.981707  11.030617       13.0        9.0       19.0   
    3          50.171429  11.853234       13.0        9.0       16.5   
1   1          38.857143  10.415190       16.0       10.5       18.0   

                  eso_LA  
                <lambda>  
sex bmi_group             
0   1          70.833333  
    2          69.480519  
    3          75.862069  
1   1          42.857143

Để slicing chỉ ở cấp ngoài cùng, bạn làm như sau.

In [10]:
# Truy cập vào sex = 1
d_agg.loc[1]

age            fssg_total                           eso_LA
                mean        std     median <lambda_0> <lambda_1>   <lambda>
bmi_group                                                                  
1          38.857143  10.415190       16.0       10.5       18.0  42.857143
2          46.250000  11.998058       12.0        9.0       16.0  52.727273
3          44.487179  10.101776       11.0        7.0       15.5  38.888889

In [11]:
# Nếu vẫn muốn giữa cấp `sex`
d_agg.loc[[1]]

age            fssg_total                        \
                    mean        std     median <lambda_0> <lambda_1>   
sex bmi_group                                                          
1   1          38.857143  10.415190       16.0       10.5       18.0   
    2          46.250000  11.998058       12.0        9.0       16.0   
    3          44.487179  10.101776       11.0        7.0       15.5   

                  eso_LA  
                <lambda>  
sex bmi_group             
1   1          42.857143  
    2          52.727273  
    3          38.888889

In [12]:
# Truy cập vào age
d_agg.loc[:, "age"]

mean        std
sex bmi_group                      
0   1          48.040000  12.921171
    2          46.981707  11.030617
    3          50.171429  11.853234
1   1          38.857143  10.415190
    2          46.250000  11.998058
    3          44.487179  10.101776

Để slice những cấp bên trong, chúng ta đặt `slice(None)` cho cấp bên ngoài.

In [13]:
d_agg.loc[(slice(None), [2, 3]), (slice(None), ["mean", "median"])]

age fssg_total
                    mean     median
sex bmi_group                      
0   2          46.981707       13.0
    3          50.171429       13.0
1   2          46.250000       12.0
    3          44.487179       11.0

Hoặc sử dụng chức năng "cắt ngang" trong Pandas bằng hàm `xs()`.

In [14]:
d_agg.xs(2, level="bmi_group", axis=0)

age            fssg_total                           eso_LA
          mean        std     median <lambda_0> <lambda_1>   <lambda>
sex                                                                  
0    46.981707  11.030617       13.0        9.0       19.0  69.480519
1    46.250000  11.998058       12.0        9.0       16.0  52.727273

## Các thao tác khác trên index nhiều cấp

### Thay đổi thứ tự các cấp

Bạn có tráo đổi thứ tự các cấp của index nhiều cấp với `swaplevel()`.

In [15]:
d_agg.swaplevel(0, 1, axis=0)

age            fssg_total                        \
                    mean        std     median <lambda_0> <lambda_1>   
bmi_group sex                                                          
1         0    48.040000  12.921171       14.0       10.0       18.0   
2         0    46.981707  11.030617       13.0        9.0       19.0   
3         0    50.171429  11.853234       13.0        9.0       16.5   
1         1    38.857143  10.415190       16.0       10.5       18.0   
2         1    46.250000  11.998058       12.0        9.0       16.0   
3         1    44.487179  10.101776       11.0        7.0       15.5   

                  eso_LA  
                <lambda>  
bmi_group sex             
1         0    70.833333  
2         0    69.480519  
3         0    75.862069  
1         1    42.857143  
2         1    52.727273  
3         1    38.888889

Hoặc dùng `reorder_levels()`.

In [16]:
d_agg.reorder_levels([1, 0], axis=1)

mean        std     median <lambda_0> <lambda_1>  \
                     age        age fssg_total fssg_total fssg_total   
sex bmi_group                                                          
0   1          48.040000  12.921171       14.0       10.0       18.0   
    2          46.981707  11.030617       13.0        9.0       19.0   
    3          50.171429  11.853234       13.0        9.0       16.5   
1   1          38.857143  10.415190       16.0       10.5       18.0   
    2          46.250000  11.998058       12.0        9.0       16.0   
    3          44.487179  10.101776       11.0        7.0       15.5   

                <lambda>  
                  eso_LA  
sex bmi_group             
0   1          70.833333  
    2          69.480519  
    3          75.862069  
1   1          42.857143  
    2          52.727273  
    3          38.888889

### Đổi tên cho cấp

Do index có nhiều cấp, danh sách tên bạn cung cấp phải có số phần tử bằng với số cấp.

In [17]:
d_agg.rename_axis(index=["Tuổi", "Giới"])

age            fssg_total                           eso_LA
                mean        std     median <lambda_0> <lambda_1>   <lambda>
Tuổi Giới                                                                  
0    1     48.040000  12.921171       14.0       10.0       18.0  70.833333
     2     46.981707  11.030617       13.0        9.0       19.0  69.480519
     3     50.171429  11.853234       13.0        9.0       16.5  75.862069
1    1     38.857143  10.415190       16.0       10.5       18.0  42.857143
     2     46.250000  11.998058       12.0        9.0       16.0  52.727273
     3     44.487179  10.101776       11.0        7.0       15.5  38.888889

### Sắp xếp danh sách chỉ mục

Bạn nên sắp xếp index nhiều cấp theo thứ tự để việc truy cập được tối ưu hóa.

In [18]:
d_agg.swaplevel(0, 1, axis=0).sort_index()

age            fssg_total                        \
                    mean        std     median <lambda_0> <lambda_1>   
bmi_group sex                                                          
1         0    48.040000  12.921171       14.0       10.0       18.0   
          1    38.857143  10.415190       16.0       10.5       18.0   
2         0    46.981707  11.030617       13.0        9.0       19.0   
          1    46.250000  11.998058       12.0        9.0       16.0   
3         0    50.171429  11.853234       13.0        9.0       16.5   
          1    44.487179  10.101776       11.0        7.0       15.5   

                  eso_LA  
                <lambda>  
bmi_group sex             
1         0    70.833333  
          1    42.857143  
2         0    69.480519  
          1    52.727273  
3         0    75.862069  
          1    38.888889

### Đổi tên các phần tử trong index

Do index có nhiều cấp, nếu muốn đổi tên các phần tử trong một cấp, bạn cần xác định rõ cấp nào muốn đổi tên trong hàm `rename()`.

In [19]:
d_agg.rename(index={1: "Thiếu cân", 2: "Bình thường", 3: "Thừa cân"}, level=1)

age            fssg_total                        \
                      mean        std     median <lambda_0> <lambda_1>   
sex bmi_group                                                            
0   Thiếu cân    48.040000  12.921171       14.0       10.0       18.0   
    Bình thường  46.981707  11.030617       13.0        9.0       19.0   
    Thừa cân     50.171429  11.853234       13.0        9.0       16.5   
1   Thiếu cân    38.857143  10.415190       16.0       10.5       18.0   
    Bình thường  46.250000  11.998058       12.0        9.0       16.0   
    Thừa cân     44.487179  10.101776       11.0        7.0       15.5   

                    eso_LA  
                  <lambda>  
sex bmi_group               
0   Thiếu cân    70.833333  
    Bình thường  69.480519  
    Thừa cân     75.862069  
1   Thiếu cân    42.857143  
    Bình thường  52.727273  
    Thừa cân     38.888889

Trong trường hợp muốn đổi tên cả hai cấp, bạn không cần cung cấp đối số `level`, và từ điển đổi tên của bạn sẽ được áp dụng cho tất cả các cấp.

In [20]:
replace_dict = {
    "age": "Tuổi",
    "fssg_total": "FSSG (Tổng)",
    "eso_LA": "Viêm thực quản",
    "std": "SD",
    "<lambda_0>": "Q1",
    "<lambda_1>": "Q3",
    "<lambda>": "%",
}

d_agg.rename(columns=replace_dict)

Tuổi            FSSG (Tổng)             Viêm thực quản
                    mean         SD      median    Q1    Q3              %
sex bmi_group                                                             
0   1          48.040000  12.921171        14.0  10.0  18.0      70.833333
    2          46.981707  11.030617        13.0   9.0  19.0      69.480519
    3          50.171429  11.853234        13.0   9.0  16.5      75.862069
1   1          38.857143  10.415190        16.0  10.5  18.0      42.857143
    2          46.250000  11.998058        12.0   9.0  16.0      52.727273
    3          44.487179  10.101776        11.0   7.0  15.5      38.888889

### Nén các cấp thành một cấp

Trong một số trường hợp, bạn muốn chuyển index nhiều cấp thành một cấp bằng cách gộp các giá trị ở các cấp lại với nhau. Pandas cung cấp hàm `to_flat_index()` cho việc này.

In [21]:
d_agg.columns.to_flat_index()

Index([             ('age', 'mean'),               ('age', 'std'),
           ('fssg_total', 'median'), ('fssg_total', '<lambda_0>'),
       ('fssg_total', '<lambda_1>'),       ('eso_LA', '<lambda>')],
      dtype='object')

Hoặc bạn có thể dùng vòng lặp để tạo ra index mới.

In [22]:
pd.Index(map(lambda x: "{} ({})".format(*x), d_agg.columns))

Index(['age (mean)', 'age (std)', 'fssg_total (median)',
       'fssg_total (<lambda_0>)', 'fssg_total (<lambda_1>)',
       'eso_LA (<lambda>)'],
      dtype='object')

### Groupby với index nhiều cấp

Như đã chia sẻ trong một ví dụ ở bài [D10](./10_reshape.ipynb), bạn có thể cung cấp index cho `groupby()` để thực hiện việc nhóm. Một tính năng của Groupby mà mình chưa giới thiệu trong các bài trước là lọc (filter) trong các nhóm và sau đó trả về một data frame kết quả lọc tổng hợp. Chẳng hạn, bạn có thể lấy dòng đầu tiên của mỗi giới tính như sau.

In [23]:
d_agg.groupby(level=0).head(1)

age            fssg_total                        \
                    mean        std     median <lambda_0> <lambda_1>   
sex bmi_group                                                          
0   1          48.040000  12.921171       14.0       10.0       18.0   
1   1          38.857143  10.415190       16.0       10.5       18.0   

                  eso_LA  
                <lambda>  
sex bmi_group             
0   1          70.833333  
1   1          42.857143

Bạn có thể làm tương tự như vậy với các cột.

In [24]:
d_agg.reset_index().groupby("bmi_group").head(1)

sex bmi_group        age            fssg_total                        \
                      mean        std     median <lambda_0> <lambda_1>   
0   0         1  48.040000  12.921171       14.0       10.0       18.0   
1   0         2  46.981707  11.030617       13.0        9.0       19.0   
2   0         3  50.171429  11.853234       13.0        9.0       16.5   

      eso_LA  
    <lambda>  
0  70.833333  
1  69.480519  
2  75.862069

## Tối ưu hóa việc sử dụng index nhiều cấp

Với ví dụ aggregation ở trên, bạn sẽ thấy rằng mỗi biến `"age"`, `"fssg_total"` và `"eso_LA"` được thống kê theo một cách khác nhau. Chúng ta rất khó thao tác tự động trên những tổ hợp index như vậy. Bạn nên lập kế hoạch để phân tích tự động kết hợp với index nhiều cấp một cách thuận lợi hơn.

Trong ví dụ dưới đây, mình tạo ra ba hàm `get_mean_sd()`, `get_median_iqr()`, và `get_prop()` để thống kê lần lượt mean (SD) và median (Q1, Q3) cho biến định lượng và n (%) cho biến định tính. Sau đó, mình kết hợp ba hàm này trong hàm `table1()` để lập ra một bảng 1 chia theo nhóm. Nếu bạn bị "hoa mắt" bởi những dòng code dưới đây, bạn có thể chạy riêng từng dòng code và xem kết quả của hàm, và bạn cũng nên xem lại các bài trước trong chương Pandas này.

In [25]:
def get_mean_sd(d: pd.DataFrame, groupby: list, cols: list) -> pd.DataFrame:
    d_agg = d.groupby(groupby)[cols] \
        .agg(["mean", "std"]) \
        .stack(0).apply(lambda x: "{:.2f} ({:.2f})".format(*x), axis=1) \
        .unstack(-1).T
    d_agg.index = pd.MultiIndex.from_arrays([d_agg.index, [""] * d_agg.index.shape[0]], names=["variable", "value"])
    return d_agg

def get_median_iqr(d: pd.DataFrame, groupby: list, cols: list) -> pd.DataFrame:
    d_agg = d.groupby(groupby)[cols] \
        .agg(["median", lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)]) \
        .stack(0).apply(lambda x: "{:.1f} ({:.1f}, {:.1f})".format(*x), axis=1) \
        .unstack(-1).T
    d_agg.index = pd.MultiIndex.from_arrays([d_agg.index, [""] * d_agg.index.shape[0]], names=["variable", "value"])
    return d_agg

def get_prop(d: pd.DataFrame, groupby: list, cols: list) -> pd.DataFrame:
    return d[groupby + cols].melt(id_vars=groupby).dropna() \
        .groupby(groupby + ["variable", "value"]) \
        .agg(lambda x: x.shape[0]).rename("n").reset_index() \
        .assign(
            total = lambda x: x.groupby(groupby + ["variable"])["n"].transform("sum"),
            pct = lambda x: x["n"] / x["total"] * 100,
            n_pct = lambda x: x[["n", "pct"]].apply(lambda y: "{:.0f} ({:.1f})".format(*y), axis=1)
        ) \
        .drop(columns=["n", "total", "pct"]) \
        .pivot(index=["variable", "value"], columns=groupby, values="n_pct")

def table1(d: pd.DataFrame, groupby, cols_all: list, cols_mean: list, cols_median: list) -> pd.DataFrame:
    cols_prop = list(set(cols_all).difference(cols_mean + cols_median))
    if type(groupby) is not list:
        groupby = [groupby]

    return pd.concat([
        get_prop(d, groupby, cols_prop),
        get_mean_sd(d, groupby, cols_mean),
        get_median_iqr(d, groupby, cols_median)
    ]) \
        .fillna("-") \
        .reindex(index=cols_all, level="variable")

groupby = ["sex", "bmi_group"]
cols_all = ["age", "eso_LA", "hp_endo", "les_irp4s", "les_baserestp", "fssg_total"]
cols_mean = ["les_irp4s", "les_baserestp"]
cols_median = ["age", "fssg_total"]
cols_prop = ["eso_LA", "hp_endo"]
table1(d, groupby, cols_all, cols_mean, cols_median)

sex                                  0                                        \
bmi_group                            1                  2                  3   
variable      value                                                            
age                  37.0 (56.0, 48.0)  38.0 (55.0, 46.0)  41.5 (57.5, 52.0)   
eso_LA        0.0            17 (70.8)         107 (69.5)          22 (75.9)   
              1.0             7 (29.2)          45 (29.2)           7 (24.1)   
              2.0                    -            1 (0.6)                  -   
              3.0                    -            1 (0.6)                  -   
hp_endo       0.0            16 (66.7)         115 (75.7)          22 (78.6)   
              1.0             8 (33.3)          37 (24.3)           6 (21.4)   
les_irp4s                  7.50 (5.53)        5.93 (4.52)        5.71 (3.91)   
les_baserestp            23.05 (13.09)       18.06 (7.95)       19.24 (7.24)   
fssg_total           10.0 (18.0, 14.0)   9.0 (19.0, 13.0)   9.0 (16.5, 13.0)   

sex                                  1                                        
bmi_group                            1                  2                  3  
variable      value                                                           
age                  30.5 (43.5, 38.0)  37.0 (54.2, 44.5)  37.0 (52.0, 45.0)  
eso_LA        0.0             3 (42.9)          29 (52.7)          14 (38.9)  
              1.0             4 (57.1)          22 (40.0)          15 (41.7)  
              2.0                    -            3 (5.5)           6 (16.7)  
              3.0                    -            1 (1.8)            1 (2.8)  
hp_endo       0.0             5 (71.4)          45 (81.8)          26 (72.2)  
              1.0             2 (28.6)          10 (18.2)          10 (27.8)  
les_irp4s                  4.59 (4.75)        4.99 (4.32)        4.07 (4.25)  
les_baserestp             13.20 (5.13)      15.71 (10.52)       13.48 (6.99)  
fssg_total           10.5 (18.0, 16.0)   9.0 (16.0, 12.0)   7.0 (15.5, 11.0)

Thêm một chút gia vị cuối cùng: kết hợp với câu chuyện data dictionary hôm trước, chúng ta hãy tạo ra một bảng 1 hoàn chỉnh.

In [26]:
# Thiết kế nhãn
var_labels = ["Giới", "BMI", "Tuổi", "Viêm thực quản trào ngược",
    "Kết quả H. pylori nội soi", "IRP 4s", "Áp lực LES khi nghỉ", "Tổng điểm FSSG"]
var_label_dict = dict(zip(groupby + cols_all, var_labels))

replace_labels = [
    {0: "Nữ", 1: "Nam"},
    {1: "Thiếu cân", 2: "Bình thường", 3: "Thừa cân"},
    {0: "Không", 1: "LA A", 2: "LA B", 3: "LA C"},
    {0: "Âm tính", 1: "Dương tính"}
]
replace_dict = dict(zip(groupby + cols_prop, replace_labels))

order_labels = [
    ["Nam", "Nữ"],
    ["Bình thường", "Thiếu cân", "Thừa cân"],
    ["Không", "LA A", "LA B", "LA C"],
    ["Dương tính", "Âm tính"]
]
order_dict = dict(zip(groupby + cols_prop, order_labels))

# Dán nhãn và chuyển sang dữ liệu categorical
d_label = d[groupby + cols_all].replace(replace_dict)
for v, order in order_dict.items():
    d_label[v] = pd.Categorical(d_label[v], categories=order, ordered=True)

# Viết lại hàm table1() để thêm dán nhãn
def table1(d: pd.DataFrame, groupby, cols_all: list, cols_mean: list, cols_median: list, var_label_dict: dict) -> pd.DataFrame:
    cols_prop = list(set(cols_all).difference(cols_mean + cols_median))
    if type(groupby) is not list:
        groupby = [groupby]

    return pd.concat([
        get_prop(d, groupby, cols_prop),
        get_mean_sd(d, groupby, cols_mean),
        get_median_iqr(d, groupby, cols_median)
    ]) \
        .fillna("-") \
        .reindex(index=cols_all, level="variable") \
        .rename(index=var_label_dict, level="variable") \
        .rename_axis(
            columns=[var_label_dict[v] for v in groupby],
            index=["Đặc điểm", "Nhóm"]
        )

table1(d_label, groupby, cols_all, cols_mean, cols_median, var_label_dict)

Giới                                                Nam                     \
BMI                                         Bình thường          Thiếu cân   
Đặc điểm                  Nhóm                                               
Tuổi                                  37.0 (54.2, 44.5)  30.5 (43.5, 38.0)   
Viêm thực quản trào ngược Không               29 (52.7)           3 (42.9)   
                          LA A                22 (40.0)           4 (57.1)   
                          LA B                  3 (5.5)                  -   
                          LA C                  1 (1.8)                  -   
Kết quả H. pylori nội soi Dương tính          10 (18.2)           2 (28.6)   
                          Âm tính             45 (81.8)           5 (71.4)   
IRP 4s                                      4.99 (4.32)        4.59 (4.75)   
Áp lực LES khi nghỉ                       15.71 (10.52)       13.20 (5.13)   
Tổng điểm FSSG                         9.0 (16.0, 12.0)  10.5 (18.0, 16.0)   

Giới                                                                    Nữ  \
BMI                                            Thừa cân        Bình thường   
Đặc điểm                  Nhóm                                               
Tuổi                                  37.0 (52.0, 45.0)  38.0 (55.0, 46.0)   
Viêm thực quản trào ngược Không               14 (38.9)         107 (69.5)   
                          LA A                15 (41.7)          45 (29.2)   
                          LA B                 6 (16.7)            1 (0.6)   
                          LA C                  1 (2.8)            1 (0.6)   
Kết quả H. pylori nội soi Dương tính          10 (27.8)          37 (24.3)   
                          Âm tính             26 (72.2)         115 (75.7)   
IRP 4s                                      4.07 (4.25)        5.93 (4.52)   
Áp lực LES khi nghỉ                        13.48 (6.99)       18.06 (7.95)   
Tổng điểm FSSG                         7.0 (15.5, 11.0)   9.0 (19.0, 13.0)   

Giới                                                                        
BMI                                           Thiếu cân           Thừa cân  
Đặc điểm                  Nhóm                                              
Tuổi                                  37.0 (56.0, 48.0)  41.5 (57.5, 52.0)  
Viêm thực quản trào ngược Không               17 (70.8)          22 (75.9)  
                          LA A                 7 (29.2)           7 (24.1)  
                          LA B                        -                  -  
                          LA C                        -                  -  
Kết quả H. pylori nội soi Dương tính           8 (33.3)           6 (21.4)  
                          Âm tính             16 (66.7)          22 (78.6)  
IRP 4s                                      7.50 (5.53)        5.71 (3.91)  
Áp lực LES khi nghỉ                       23.05 (13.09)       19.24 (7.24)  
Tổng điểm FSSG                        10.0 (18.0, 14.0)   9.0 (16.5, 13.0)

---

[Bài trước](./16_datadict.ipynb) - [Danh sách bài](../README.md) - [Bài sau](../05_regex/01_basic.ipynb)